In [1]:
import torch
from codes.model import  ResUnet, Loss_With_Weight
from torch.utils.data import DataLoader
from codes.dataloader import init_dataset, STEAD_Dataset
from codes.visualize_and_evaluate import evaluate
from codes import configs

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

train_set,val_set,noise_set = init_dataset()
train_set = STEAD_Dataset(train_set)
val_set = STEAD_Dataset(val_set)

train_dataloader = DataLoader(train_set, shuffle=True, batch_size=configs.BATCH_SIZE, num_workers=4, drop_last=True)
eval_dataloader = DataLoader(val_set, shuffle=True,batch_size=1, num_workers=4, drop_last=True)

mseLoss = torch.nn.MSELoss()
QuackNet = ResUnet().to(device)

optimizer = torch.optim.Adam(QuackNet.parameters(),
                lr=configs.LEARNING_RATE,
                betas=(0.9, 0.999),
                eps=1e-08,
                weight_decay=0,
                amsgrad=False)

total_step = 0
loss_sum = 0
loss_list = []
for epoch in range(configs.EPOCH_NUM):
    # training
    for index,data in enumerate(train_dataloader,0):
        stream,label_p,label_s,p_start,s_start,coda_end = data

        # 清空累加梯度
        QuackNet.zero_grad()
        output = QuackNet(stream.to(device))
        loss = 0
        loss += Loss_With_Weight(output[:,0,:], label_p.to(device))
        loss += Loss_With_Weight(output[:,1,:], label_s.to(device))
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
        total_step += 1
        if total_step%200 == 0:
            loss_list.append({
                "total_step":total_step,
                "loss:":loss
            })
            print("total_step:{} loss:{}".format(total_step,loss_sum/200))
            loss_sum = 0

        if total_step%configs.CHECKPOINT_ITER == 0:
            torch.save(QuackNet.eval(),configs.model_save_dir+"QuackPicker_iter{}.pth".format(total_step))
            evaluate(configs.model_save_dir+"QuackPicker_iter{}.pth".format(total_step),"batchsize:{} steps:{}".format(configs.BATCH_SIZE,total_step),eval_dataloader,50,QuackNet)
        

C:\Users\29147\.conda\envs\pytorch\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (7,11,13,14,15,18,19,20,21,22,24,25,26,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


total events in csv file: 1265657
total events selected: 574007


C:\Users\29147\.conda\envs\pytorch\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (7,11,13,14,15,18,19,20,21,22,24,25,26,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


total noise selected: 235426
torch.Size([32, 24, 25])


C:\Users\29147\.conda\envs\pytorch\lib\site-packages\torch\nn\functional.py:3458: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


torch.Size([32, 24, 25])
torch.Size([32, 24, 25])
torch.Size([32, 24, 25])
torch.Size([32, 24, 25])
torch.Size([32, 24, 25])
torch.Size([32, 24, 25])
torch.Size([32, 24, 25])
torch.Size([32, 24, 25])
torch.Size([32, 24, 25])
torch.Size([32, 24, 25])
torch.Size([32, 24, 25])
torch.Size([32, 24, 25])
torch.Size([32, 24, 25])
torch.Size([32, 24, 25])
torch.Size([32, 24, 25])
torch.Size([32, 24, 25])
torch.Size([32, 24, 25])


In [4]:

evaluate(configs.model_save_dir+"QuackPicker_iter{}.pth".format(total_step),"batchsize:{} steps:{}".format(configs.BATCH_SIZE,total_step),eval_dataloader,50,20)

Evaluating...    100%

AttributeError: 'Evaluator' object has no attribute 'merge_evaluator'

In [1]:
import random
import torch
import torch.nn as nn
import torch.nn.functional as F

In [19]:
random.randint(9,13)

11

In [2]:
lstm = nn.LSTM(3, 3)